In [1]:
"""
train the objective function, using interatomic distance distributions that are computed
from a dataset of known (
i.e., experimentally determined) 3D structures;
- plot the scoring profiles,
i.e. the score (or estimated Gibbs free energy) as a function of
the interatomic distance;
- use the objective function to evaluate predicted structures from the RNA-Puzzles
dataset
"""

'\ntrain the objective function, using interatomic distance distributions that are computed\nfrom a dataset of known (\ni.e., experimentally determined) 3D structures;\n- plot the scoring profiles,\ni.e. the score (or estimated Gibbs free energy) as a function of\nthe interatomic distance;\n- use the objective function to evaluate predicted structures from the RNA-Puzzles\ndataset\n'

In [1]:
# Developing the objective(scoring) function based on known RNA 3D structures
import os
import sys
import argparse
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms


In [ ]:
from Bio.PDB import MMCIFParser, PDBParser

# create parser
MMCIFparser = MMCIFParser(QUIET=True)  # QUIET=True 
PDBparser = PDBParser(QUIET=True)

# read mmCIF or PDB file
#structure = MMCIFparser.get_structure("example", "./data/6e4o.cif")
structure = PDBparser.get_structure("example", "./data/6e4o.pdb")

# traverse structure
for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                print(atom.get_name(), atom.get_coord())

In [ ]:
from rdkit import Chem

class Residue:
    def __init__(self, resname, resseq, atoms=None):
        self.resname = resname      # Residue name, e.g., 'ALA' or 'LIG' if exists
        self.resseq = resseq        # Residue sequence number
        self.atoms = atoms or {}    # {'C3': coord, ...}
        self.coarsed_atoms = {}  # {'C3': coord, ...} for coarse-grained atoms, C3 default

class Chain:
    def __init__(self, chain_id):
        self.chain_id = chain_id
        self.residues = []          # Residue object list

class Rna:
    def __init__(self, pdb_id):
        self.pdb_id = pdb_id
        self.chains = {}            # Chain object list

# RDKit
class Ligand:
    def __init__(self, mol_file):
        self.mol = Chem.MolFromMol2File(mol_file)  # RDKit Mol object

In [8]:
# traverse structure to classes
for model in structure:
    rna = Rna(pdb_id="example")
    for chain in model:
        residues = Chain(chain_id=chain.id)
        for residue in chain:
            res = Residue(resname=residue.get_resname(), resseq=residue.get_id()[1])
            for atom in residue:
                atom_name = atom.get_name()
                if atom_name == "C":
                    atom_name = "C3"
                res.atoms[atom_name] = atom.get_coord()
            residues.residues.append(res)
        rna.chains[chain.id] = residues

In [50]:
from Bio.PDB import MMCIFParser, PDBIO, Select

class LigandSelect(Select):
    def accept_residue(self, residue):
        # 保留非标准残基（HETATM），排除水
        return residue.id[0] != " " and residue.resname != "HOH"

parser = MMCIFParser(QUIET=True)
structure = parser.get_structure("example", "./data/6e4o.cif")

io = PDBIO()
io.set_structure(structure)
io.save("ligand_only.pdb", LigandSelect()) # extract ligands only

In [51]:
!obabel ligand_only.pdb -O ligand.sdf # generate sdf ligand file

0 molecules converted


In [34]:
from rdkit import Chem # ligands RDKit
ligand_mol = Chem.MolFromMolFile("ligand.sdf")

In [32]:
"""
for chain in rna.chains:
    for residue in rna.chains[chain].residues:
        if 'C3' in residue.atoms:
            print(residue.atoms['C3'])
"""

"\nfor chain in rna.chains:\n    for residue in rna.chains[chain].residues:\n        if 'C3' in residue.atoms:\n            print(residue.atoms['C3'])\n"

In [ ]:
### Coarsed grained representation
Coarsed = 